In [10]:
!pip install groq requests python-dotenv

In [11]:
from groq import Groq
import json
import time
from typing import Dict, List

# Initialize Groq client with your API key
GROQ_API_KEY = ""
client = Groq(api_key=GROQ_API_KEY)

# We'll use llama-3.3-70b-versatile for our agents
MODEL = "llama-3.3-70b-versatile"

print("✅ Setup complete! Multi-agent system ready.")

✅ Setup complete! Multi-agent system ready.


In [12]:
class Agent:
    """Base class for all agents in our multi-agent system"""

    def __init__(self, name: str, role: str, expertise: str):
        self.name = name
        self.role = role
        self.expertise = expertise
        self.memory = []

    def think(self, task: str, context: str = "") -> str:
        """Agent thinks about the task using Groq API"""

        system_prompt = f"""You are {self.name}, a {self.role}.
Your expertise: {self.expertise}
Always respond in a helpful, focused manner based on your role."""

        user_message = f"Task: {task}"
        if context:
            user_message += f"\n\nContext from other agents:\n{context}"

        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            temperature=0.7,
            max_tokens=500
        )

        result = response.choices[0].message.content
        self.memory.append({"task": task, "response": result})
        return result

    def __str__(self):
        return f"🤖 {self.name} ({self.role})"

print("✅ Agent base class defined!")

✅ Agent base class defined!


In [13]:
# Agent 1: Researcher - Gathers information and facts
researcher = Agent(
    name="ResearchBot",
    role="Research Specialist",
    expertise="Finding relevant information, gathering facts, and identifying key points"
)

# Agent 2: Analyzer - Analyzes data and provides insights
analyzer = Agent(
    name="AnalyzerBot",
    role="Data Analyst",
    expertise="Analyzing information, identifying patterns, and drawing conclusions"
)

# Agent 3: Writer - Creates final output
writer = Agent(
    name="WriterBot",
    role="Content Creator",
    expertise="Creating clear, engaging written content based on research and analysis"
)

agents = [researcher, analyzer, writer]

print("✅ Three agents created:")
for agent in agents:
    print(f"   {agent}")

✅ Three agents created:
   🤖 ResearchBot (Research Specialist)
   🤖 AnalyzerBot (Data Analyst)
   🤖 WriterBot (Content Creator)


In [14]:
class MultiAgentOrchestrator:
    """Coordinates multiple agents to work together on complex tasks"""

    def __init__(self, agents: List[Agent]):
        self.agents = agents
        self.workflow_history = []

    def execute_workflow(self, task: str) -> Dict:
        """Execute a multi-agent workflow"""

        print(f"\n{'='*60}")
        print(f"🎯 TASK: {task}")
        print(f"{'='*60}\n")

        results = {}
        context = ""

        # Step 1: Researcher gathers information
        print(f"\n{self.agents[0]} is researching...")
        research_result = self.agents[0].think(task)
        results['research'] = research_result
        context = f"Research findings:\n{research_result}"
        print(f"✓ Research complete!\n")
        time.sleep(1)

        # Step 2: Analyzer analyzes the research
        print(f"\n{self.agents[1]} is analyzing...")
        analysis_task = f"Analyze the following research and provide key insights: {task}"
        analysis_result = self.agents[1].think(analysis_task, context)
        results['analysis'] = analysis_result
        context += f"\n\nAnalysis:\n{analysis_result}"
        print(f"✓ Analysis complete!\n")
        time.sleep(1)

        # Step 3: Writer creates final output
        print(f"\n{self.agents[2]} is writing...")
        writing_task = f"Create a comprehensive summary about: {task}"
        final_result = self.agents[2].think(writing_task, context)
        results['final_output'] = final_result
        print(f"✓ Writing complete!\n")

        self.workflow_history.append({
            'task': task,
            'results': results
        })

        return results

    def display_results(self, results: Dict):
        """Display the results in a formatted way"""
        print(f"\n{'='*60}")
        print("📊 MULTI-AGENT WORKFLOW RESULTS")
        print(f"{'='*60}\n")

        print(f"🔍 RESEARCH PHASE ({self.agents[0].name}):")
        print(f"{results['research']}\n")
        print(f"{'-'*60}\n")

        print(f"📈 ANALYSIS PHASE ({self.agents[1].name}):")
        print(f"{results['analysis']}\n")
        print(f"{'-'*60}\n")

        print(f"✍️ FINAL OUTPUT ({self.agents[2].name}):")
        print(f"{results['final_output']}\n")
        print(f"{'='*60}")

# Create orchestrator
orchestrator = MultiAgentOrchestrator(agents)
print("✅ Multi-agent orchestrator ready!")

✅ Multi-agent orchestrator ready!


In [15]:
# Example 1: Research and analyze a tech topic
task1 = "Explain the benefits and challenges of using multi-agent AI systems in production"

results1 = orchestrator.execute_workflow(task1)
orchestrator.display_results(results1)


🎯 TASK: Explain the benefits and challenges of using multi-agent AI systems in production


🤖 ResearchBot (Research Specialist) is researching...
✓ Research complete!


🤖 AnalyzerBot (Data Analyst) is analyzing...
✓ Analysis complete!


🤖 WriterBot (Content Creator) is writing...
✓ Writing complete!


📊 MULTI-AGENT WORKFLOW RESULTS

🔍 RESEARCH PHASE (ResearchBot):
**Introduction to Multi-Agent AI Systems**

Multi-agent AI systems involve the interaction of multiple autonomous agents, each with their own goals, behaviors, and decision-making processes. These systems can be applied to various domains, including production, logistics, and finance. The benefits and challenges of using multi-agent AI systems in production are numerous and complex.

**Benefits of Multi-Agent AI Systems in Production**

1. **Improved Efficiency**: Multi-agent systems can optimize production workflows by allocating tasks to agents with the necessary skills and resources, reducing processing time and increasin

In [16]:
# View the conversation flow between agents
def show_agent_memory():
    print("\n🧠 AGENT MEMORY & COLLABORATION:\n")
    for agent in agents:
        print(f"\n{agent}")
        print("-" * 40)
        if agent.memory:
            last_interaction = agent.memory[-1]
            print(f"Last task: {last_interaction['task'][:100]}...")
            print(f"Response length: {len(last_interaction['response'])} characters")
        else:
            print("No interactions yet")
    print()

show_agent_memory()


🧠 AGENT MEMORY & COLLABORATION:


🤖 ResearchBot (Research Specialist)
----------------------------------------
Last task: Explain the benefits and challenges of using multi-agent AI systems in production...
Response length: 2785 characters

🤖 AnalyzerBot (Data Analyst)
----------------------------------------
Last task: Analyze the following research and provide key insights: Explain the benefits and challenges of usin...
Response length: 2955 characters

🤖 WriterBot (Content Creator)
----------------------------------------
Last task: Create a comprehensive summary about: Explain the benefits and challenges of using multi-agent AI sy...
Response length: 2964 characters



In [17]:
# View workflow history and statistics
def show_workflow_stats():
    print("\n📈 WORKFLOW STATISTICS\n")
    print(f"Total workflows executed: {len(orchestrator.workflow_history)}")
    print(f"Total agent interactions: {sum(len(agent.memory) for agent in agents)}")

    print("\n📋 Workflow History:")
    for i, workflow in enumerate(orchestrator.workflow_history, 1):
        print(f"\n{i}. {workflow['task']}")
        print(f"   - Research length: {len(workflow['results']['research'])} chars")
        print(f"   - Analysis length: {len(workflow['results']['analysis'])} chars")
        print(f"   - Final output length: {len(workflow['results']['final_output'])} chars")

show_workflow_stats()


📈 WORKFLOW STATISTICS

Total workflows executed: 1
Total agent interactions: 3

📋 Workflow History:

1. Explain the benefits and challenges of using multi-agent AI systems in production
   - Research length: 2785 chars
   - Analysis length: 2955 chars
   - Final output length: 2964 chars
